In [ ]:
import pandas as pd
import altair as alt

### Let's prepare the data

In [ ]:
df = pd.read_excel(
     "2014-2020.xlsx",
     engine='openpyxl')
df.head()

In [ ]:
df.columns = ["date", "aes", "tec", "vde", "tes", "ges", "gaes",
              "consumption", "pump", "UK_BLR_RUS", "UK_EURO", "UK_MLD",
             "d1", "d2"]

df.drop(['pump', 'd1', 'd2'], axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
alt.data_transformers.disable_max_rows()

## Task 1
### How has the structure of electricity generation changed over the years?


In [ ]:
over_year = df.drop(["consumption", "UK_BLR_RUS", "UK_EURO", "UK_MLD"], axis = 1)
over_year.head()

In [ ]:
over_year = over_year.melt(id_vars=['date'],
                           var_name='source',
                           value_name='production')
over_year

In [ ]:
over_year["date"] = [int(entry.split(".")[-1]) for entry in over_year["date"]]

aggregation_functions = {'production': 'sum'}
over_year = over_year.groupby(["date", "source"], as_index=False).aggregate(aggregation_functions)
over_year.head()

In [ ]:
over_year.shape

In [ ]:
alt.Chart(over_year).mark_area().encode(
    x = alt.X('date:Q'),
    y = alt.Y('production:Q', aggregate = 'sum', stack = 'normalize'),
    color = alt.Color('source:N'),
    order = alt.Order('production:Q', aggregate='sum', sort = 'ascending')
).properties(background = '#F9F9F9', padding = 25)

### Additional data processing


In [ ]:
df.head()

In [ ]:
def p(e: str) -> str:
    h, d = e.split("-")
    h = int(h) - 1
    return f"{h}-{d}"

df["date"] = [p(e) for e in df["date"]]
df["date"] = pd.to_datetime(df["date"], format = '%H-%d.%m.%Y')
df.head()


## Task 2
### How does electricity consumption depend on the day of the year and the time of day?

In [ ]:
alt.Chart(df).mark_rect().encode(
    x='hours(date):O',
    y='day(date):O',
    color=alt.Color('mean(consumption):Q', scale = alt.Scale(scheme = 'orangered'))
).properties(width=200, height=150)


## Task 3
### How does the generation of electricity from different sources change during the day?
Note that motivation behind code repetition is the worries that initial data frame would become too big to use it conveniently if we apply melt on it.

In [ ]:
tmp = df.drop(["consumption", "UK_BLR_RUS", "UK_EURO", "UK_MLD"], axis = 1)
tmp = tmp.melt(id_vars=['date'],
                           var_name='source',
                           value_name='production')
print(tmp.shape)
tmp.head()

In [ ]:
alt.Chart(tmp).mark_line().encode(
    x='hours(date):O',
    y = alt.Y('mean(production):Q'),
    color = alt.Color('source:N'),
).properties(width=200, height=150)

## Task 4
### How does electricity consumption change during the day in terms of months of the year and seasons?

In [ ]:
alt.Chart(df).mark_rect().encode(
    x='date(date):O',
    y='consumption:Q',
    color=alt.Color('mean(consumption):Q', scale = alt.Scale(scheme = 'orangered')),
    column=alt.Column('quarter(date):N')
).properties(width=200, height=150)

## Task 5
### How does electricity consumption change during the week?

In [ ]:
alt.Chart(df).mark_bar().encode(
    x='day(date):O',
    y=alt.Y('mean(consumption):Q'),
    color=alt.Color('mean(consumption):O', scale=alt.Scale(scheme='orangered'))
).properties(width=200, height=150)